https://livebook.manning.com/book/deep-learning-with-python/chapter-7/6

In [ ]:
# (필수) 아키텍처 유형(1,2,3) 중에서 하나를 골라 지정해야 합니다. 지정한 유형과 구현결과가 일치하지 않으면 "기타그룹"으로 간주함.
DNN_TYPE = 1

In [ ]:
SEED = 42           # random_state 지정
BATCH_SIZE = 32    # batch size 지정

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

#!pip install pydot

In [ ]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(seed, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

In [ ]:
# 학습 데이터
X_train, y_train, X_test, test_id = pd.read_pickle('/kaggle/input/kml2022f-exam/exam_data.pkl')

print(X_train.shape, X_test.shape)

In [ ]:
# 스케일링
#scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 학습 데이터를 다시 학습과 검증 데이터로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=SEED)

In [ ]:
inputs = keras.Input(shape=(X_train.shape[1], ))
x1 = keras.layers.Dense(11)(inputs)
x2 = keras.layers.Dense(11)(inputs)
x3 = keras.layers.Dense(11)(inputs)

x4 = keras.layers.Dense(5)(x1)

x5 = keras.layers.Dense(3)(x4)
x6 = keras.layers.Dense(3)(x2)

x7 = keras.layers.BatchNormalization()(inputs)

x8 = keras.layers.Concatenate()([x3, x5, x6, x7])

outputs = keras.layers.Dense(1)(x8)
model = keras.Model(inputs, outputs)

keras.utils.plot_model(model, show_shapes=False, show_layer_names=False, dpi=100, rankdir='LR')

In [ ]:
# Hyperparameter 범위지정 함수: hp.Boolean(), hp.Choice(), hp.Float(), hp.Int()

def model_fn(hp):
    model = keras.Model(inputs, outputs)
    model.compile(loss='mse',
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), # Adam # learning_rate를 바꿔도 됨
                  metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])
    return model 

In [ ]:
# Tuner 설정
tuner = kt.Hyperband(hypermodel=model_fn,  # Hyperband 많이 사용함. 바꿔도 됨
                     objective=kt.Objective('val_rmse', direction="min"),   
                     max_epochs=5,   #3~5 사이가 best  # 하나의 모형을 훈련시키는 최대 에포크 수  # 너무 크게 하면 안됨
                     hyperband_iterations=1,  # 반복 횟수 # 바꿀 필요 있음 # 너무 과도하게 ㄴㄴ
                     seed=SEED,
                     overwrite=True,
                     directory='dnn_tuning')

In [ ]:
%%time

# 하이퍼튜닝 수행: tuner.search()의 파라미터는 Keras의 model.fit() 파라미터와 동일
tuner.search(X_train, y_train, 
             validation_data=(X_valid, y_valid),
             batch_size=BATCH_SIZE, # batch size 자체는 튜닝할수 없음
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)], 
             verbose=1)

# 튜닝결과 요약
tuner.results_summary(num_trials=1)

In [ ]:
# 모델 재현시 Seed 고정 필요
reset_seeds(SEED)

# 베스트 모델 사용
model = tuner.get_best_models(1)[0]
# submission 화일명에 사용
val_rmse = model.evaluate(X_valid, y_valid, verbose=0)[1]

# 전체 데이터로 재학습
hist = model.fit(
    x=np.concatenate((X_train, X_valid)),
    y=np.concatenate((y_train, y_valid)),
    batch_size=BATCH_SIZE*2, # 튜닝할 때 사용한 값과 다른 값으로 설정하는 것이 좋을 수 있음 
    epochs=1,                # 작은값으로 설정
    shuffle=False,      
    verbose=2,
)

In [ ]:
DNN_VERSION = 4.0

# 예측값 얻기
pred = model.predict(X_test).flatten()

# submission 화일 생성
filename = f'dnn_{DNN_VERSION}_{val_rmse:.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':pred}).to_csv('submission.csv', index=False)

print(f'{filename} is ready to submit.')